In [9]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
import shutil

In [10]:
## 1. Dataset das imagens
dataset_dir = os.path.join(os.getcwd(), 'imagens')
dataset_morango_dir = os.path.join(dataset_dir, 'morango')
dataset_pessego_dir = os.path.join(dataset_dir, 'pessego')
dataset_roma_dir = os.path.join(dataset_dir, 'roma')

dataset_morango_len = len(os.listdir(dataset_morango_dir))
dataset_pessego_len = len(os.listdir(dataset_pessego_dir))
dataset_roma_len = len(os.listdir(dataset_roma_dir))

print(f'Contagem de imagens de morango: {dataset_morango_len}')
print(f'Contagem de imagens de pêssego: {dataset_pessego_len}')
print(f'Contagem de imagens de romã: {dataset_roma_len}')

Contagem de imagens de morango: 250
Contagem de imagens de pêssego: 250
Contagem de imagens de romã: 311


In [ ]:
# 2. Carregamento das imagens e pré-processamento
# -> Normalização dos pixels das imagens para o intervalo [0,1]
from PIL import Image
import os
import numpy as np

dataset_dir = os.path.join(os.getcwd(), 'imagens')
classes = ['morango', 'pessego', 'roma']

def carregar_imagens(diretorio_base, tamanho=(500, 500)):
    imagens = []
    rotulos = []
    for rotulo, pasta in enumerate(['morango', 'pessego', 'roma']):
        pasta_path = os.path.join(diretorio_base, pasta)
        for imagem_nome in os.listdir(pasta_path):
            imagem_path = os.path.join(pasta_path, imagem_nome)
            imagem = Image.open(imagem_path)
            imagem = imagem.resize(tamanho)
            imagem_array = np.array(imagem) / 255.0  # Normalizar para o intervalo [0, 1]
            imagens.append(imagem_array)
            rotulos.append(rotulo)  # Usando o índice como rótulo
    return np.array(imagens), np.array(rotulos)

X, Y = carregar_imagens(dataset_dir)
print(X)
print(Y)

In [5]:
# 3. Aplicando o rebalanceamento do dataset
# -> Utilizada a técnica de 'data augmentation' para gerar novas imagens de morango e pessego

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Criar um objeto ImageDataGenerator para data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,          # Faixa de rotação aleatória em graus
    width_shift_range=0.2,      # Faixa de deslocamento horizontal aleatório (como uma fração da largura total)
    height_shift_range=0.2,     # Faixa de deslocamento vertical aleatório (como uma fração da altura total)
    shear_range=0.2,            # Faixa de cisalhamento aleatório (em radianos)
    zoom_range=0.2,             # Faixa de zoom aleatório
    horizontal_flip=True,      # Inverter aleatoriamente as imagens horizontalmente (espelhamento)
    fill_mode='nearest'        # Estratégia de preenchimento usada para preencher novos pixels gerados após a rotação ou deslocamento
)

numero_imagens_aumentadas = 61
classes_aumentadas = ['morango', 'pessego']

for classe in classes_aumentadas:
    # Diretório da classe atual
    diretorio_classe = os.path.join(dataset_dir, classe)
    
    # Criar diretório para a classe aumentada, se não existir
    diretorio_destino_classe = os.path.join(dataset_dir, classe)
    os.makedirs(diretorio_destino_classe, exist_ok=True)
    
    # Lista de arquivos de imagem na classe atual
    imagens_classe = os.listdir(diretorio_classe)
    
    # Selecionar aleatoriamente algumas imagens existentes para data augmentation
    indices_amostra = np.random.choice(len(imagens_classe), numero_imagens_aumentadas, replace=True)
    
    # Para cada imagem de amostra selecionada
    for indice in indices_amostra:
        imagem_nome = imagens_classe[indice]
        imagem_path = os.path.join(diretorio_classe, imagem_nome)
        
        # Carregar imagem
        imagem = Image.open(imagem_path)
        imagem_array = np.array(imagem)
        imagem_array = imagem_array.reshape((1,) + imagem_array.shape)  # Reshape para (1, altura, largura, canais)
        
        # Gerar imagens aumentadas e salvar no diretório de destino
        for i, batch in enumerate(datagen.flow(imagem_array, batch_size=1)):
            if i >= 1:  # Quantidade de imagens aumentadas a serem geradas por imagem de amostra
                break
            imagem_aumentada = batch[0].astype(np.uint8)  # Converter de volta para o formato de imagem
            nova_imagem_nome = f"{os.path.splitext(imagem_nome)[0]}_aug_{i}.jpg"
            nova_imagem_path = os.path.join(diretorio_destino_classe, nova_imagem_nome)
            nova_imagem = Image.fromarray(imagem_aumentada)
            nova_imagem.save(nova_imagem_path)

In [11]:
# 4. Definição e separação dos dados de treinamento e dados de teste
projeto_dir = os.getcwd() # Diretório do projeto
dataset_treinamento_dir = os.path.join(projeto_dir, 'imagens_treinamento') # Caminho para imagens_treinamento
dataset_teste_dir = os.path.join(projeto_dir, 'imagens_teste') # Caminho para imagens_teste

proporcao_dataset_treinamento = 0.8 # 80% será utilizado no treinamento
proporcao_dataset_teste = 1 - proporcao_dataset_treinamento # 20% será utilizado no teste
random_seed = 42 # A resposta para a Vida - segundo livro de Aurélien Géron - gerar com random

classes = ['morango', 'pessego', 'roma']

# Função que executará a separação das imagens (utiliza algoritmo de aleatoriedade)
def split_dataset_to_train_and_test(classe):
    dataset_treinamento_classe_dir = os.path.join(dataset_treinamento_dir, classe) # Dir. destino treinamento classe
    dataset_teste_classe_dir = os.path.join(dataset_teste_dir, classe) # Dir. destino teste classe

    dataset_classe_dir = os.path.join(projeto_dir, 'imagens', classe) # Dir. origem imagens classe
    imagens_classe = [os.path.join(dataset_classe_dir, img) for img in os.listdir(dataset_classe_dir)] # Popular lista com imagens

    imagens_treinamento_classe, imagens_teste_classe = train_test_split(imagens_classe, test_size=proporcao_dataset_teste, random_state=random_seed) # Algoritmo que separa aleatoriamente as imagens de treinamento e de teste

    # Criação dos diretórios de treinamento e teste para a classe
    os.makedirs(dataset_treinamento_classe_dir, exist_ok=True)
    os.makedirs(dataset_teste_classe_dir, exist_ok=True)

    # Copia as imagens de treinamento para a pasta de treinamento da classe em questão
    for imagem in imagens_treinamento_classe:
        shutil.copy(imagem, dataset_treinamento_classe_dir) 

    # Copia as imagens de teste para a pasta de teste da classe em questão
    for imagem in imagens_teste_classe:
        shutil.copy(imagem, dataset_teste_classe_dir) 

# Verifica se os diretórios de treinamento e teste já existem
if not os.path.exists(dataset_treinamento_dir) or not os.path.exists(dataset_teste_dir):
    # Cria os diretórios de treinamento e teste
    os.makedirs(dataset_treinamento_dir, exist_ok=True)
    os.makedirs(dataset_teste_dir, exist_ok=True)
    # Para cada classe, executa a função de divisão de dados
    for classe in classes:
        split_dataset_to_train_and_test(classe)

In [12]:
## 1.3. Como ficou dataset de treinamento e de testes:

dataset_treinamento_morango_len = len(os.listdir(os.path.join(dataset_treinamento_dir, 'morango')))
dataset_teste_morango_len = len(os.listdir(os.path.join(dataset_teste_dir, 'morango')))
dataset_treinamento_pessego_len = len(os.listdir(os.path.join(dataset_treinamento_dir, 'pessego')))
dataset_teste_pessego_len = len(os.listdir(os.path.join(dataset_teste_dir, 'morango')))
dataset_treinamento_roma_len = len(os.listdir(os.path.join(dataset_treinamento_dir, 'roma')))
dataset_teste_roma_len = len(os.listdir(os.path.join(dataset_teste_dir, 'roma')))

print(f'Contagem de imagens de morango para treinamento: {dataset_treinamento_morango_len}')
print(f'Contagem de imagens de morango para teste: {dataset_teste_morango_len}')
print(f'Contagem de imagens de pêssego para treinamento: {dataset_treinamento_pessego_len}')
print(f'Contagem de imagens de pêssego para teste: {dataset_teste_pessego_len}')
print(f'Contagem de imagens de romã para treinamento: {dataset_treinamento_roma_len}')
print(f'Contagem de imagens de romã para teste: {dataset_teste_roma_len}')

# desbalanceamento para a classe de romã
# considerar balancear


Contagem de imagens de morango para treinamento: 200
Contagem de imagens de morango para teste: 50
Contagem de imagens de pêssego para treinamento: 200
Contagem de imagens de pêssego para teste: 50
Contagem de imagens de romã para treinamento: 248
Contagem de imagens de romã para teste: 63


In [13]:
## 2. Pré-processamento das imagens
## --> Definir tamanho de entrada das minhas imagens (em px)
## --> Definir qual estratégia de conversão adotar (scaling da imagem / foco no centro da imagem ignorando periferia / recortar imagem até
## no limite do tamanho definido e ignorar o restante)

image_width = 160
image_heigth = 160
image_size = (image_width, image_heigth)

image_color_channel = 3
image_color_channel_size = 255
image_shape = image_size + (image_color_channel,)

batch_size = 32 # Valor que vou puxar do dataset por vez
epoch = 20 # Quantidade de vezes que vou percorrer meu dataset inteiro
learning_rate = 0.0001 # Taxa de aprendizagem

classes = ['morango', 'pessego', 'roma']

data_set_treinamento = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_treinamento_dir,
    image_size = image_size,
    batch_size = batch_size,
    shuffle = True
)

data_set_teste = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_teste_dir,
    image_size = image_size,
    batch_size = batch_size,
    shuffle = True
)

Found 648 files belonging to 3 classes.
Found 163 files belonging to 3 classes.
